# Week 5_Day 5 : CIFAR-10 Tiny Model + GPU

### CONCEPT 1: CIFAR-10 Basics

**What is CIFAR-10?**

CIFAR-10 is a dataset of **60,000 color images** divided into **10 classes**:

- airplane, automobile, bird, cat, deer, dog, frog, horse, ship, truck

**Dataset Split**

- **50,000** training images  
- **10,000** test images

### CONCEPT 2: Image shapes

**In PyTorch, images usually look like this:**

=> (C, H, W)

- C = channels (RGB = 3)  
- H = height  
- W = width  

**So a CIFAR-10 image is:**

- (3, 32, 32)

**A batch is:**

- (batch_size, 3, 32, 32)

### HANDS-ON — Load CIFAR-10 and inspect shapes

In [1]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

Step 1: Transforms

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),
                         (0.2470, 0.2435, 0.2616))
])

Step 2: Datasets

In [3]:
train_dataset = datasets.CIFAR10(root="data", train=True, download=True, transform=transform)
test_dataset  = datasets.CIFAR10(root="data", train=False, download=True, transform=transform)

len(train_dataset), len(test_dataset)

(50000, 10000)

Step 3: Dataloaders

In [4]:
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

Step 4: Check One Batch

In [5]:
images, labels = next(iter(train_loader))
print("Images shape:", images.shape)
print("Labels shape:", labels.shape)

Images shape: torch.Size([64, 3, 32, 32])
Labels shape: torch.Size([64])


### Defining a Tiny Model

In [6]:
class TinyFC(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Flatten(),
            nn.Linear(3*32*32, 128),
            nn.ReLU(),
            nn.Linear(128,10)
        )

    def forward(self, x):
        return self.net(x)
    
model = TinyFC()

### CONCEPT 3: GPU usage

In [7]:
# choose device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# move model to device
model.to(device)

# move every batch to device
images = images.to(device)
labels = labels.to(device)

### Training Loop & Accuracy

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

def accuracy_on_loader(model, loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            logits = model(images)
            preds = torch.argmax(logits, dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return correct / total

epochs = 2
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        logits = model(images)
        loss = criterion(logits, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    train_loss = running_loss / len(train_loader)
    test_acc = accuracy_on_loader(model, test_loader)

    print(f"Epoch {epoch+1}/{epochs} | Train Loss: {train_loss:.4f} | Test Acc: {test_acc*100:.2f}%")

Epoch 1/2 | Train Loss: 1.6859 | Test Acc: 45.95%
Epoch 2/2 | Train Loss: 1.4710 | Test Acc: 47.91%


### CPU vs GPU speed comparison

In [9]:
import time

def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        logits = model(images)
        loss = criterion(logits, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    return running_loss / len(loader)

def time_one_epoch(on_device):
    model = TinyFC().to(on_device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    start = time.time()
    loss = train_one_epoch(model, train_loader, optimizer, criterion, on_device)
    end = time.time()

    return loss, end - start

# CPU timing
cpu_loss, cpu_time = time_one_epoch(torch.device("cpu"))
print(f"CPU  -> Loss: {cpu_loss:.4f}, Time: {cpu_time:.2f}s")

# GPU timing (if available)
if torch.cuda.is_available():
    gpu_loss, gpu_time = time_one_epoch(torch.device("cuda"))
    print(f"GPU  -> Loss: {gpu_loss:.4f}, Time: {gpu_time:.2f}s")
else:
    print("No GPU available on this machine.")


CPU  -> Loss: 1.6866, Time: 7.53s
GPU  -> Loss: 1.6789, Time: 9.82s
